# Controlling and Structuring Outputs in Google ADK
## A Practical Guide to Predictable Agent Responses

This notebook covers:
1. **Why structured outputs matter**
2. **Defining schemas with Pydantic**
3. **Generating JSON outputs with LlmAgent**
4. **Live demonstration with email and todo list examples**

Official Documentation Reference: [ADK Agents Guide](https://google.github.io/adk-docs/develop/agents/)

## 1. Importance of Structured Data

Structured outputs provide:

```mermaid
graph TD
A[Structured Outputs] --> B[Predictable Format]
A --> C[Easy Integration]
A --> D[Automatic Validation]
B --> E[Consistent API responses]
C --> F[Seamless app/workflow usage]
D --> G[Fewer errors in production]
```

### Example: Unstructured vs Structured Email

**Unstructured (Problematic)**
```
Subject: Meeting tomorrow Hi team let's meet tomorrow thanks John
```

**Structured (Ideal)**
```json
{
  "subject": "Team Meeting Tomorrow",
  "body": "Hi Team..."
}
```

Key Benefits:
- Programmatically extract fields
- Validate before processing
- Standardized across agents

## 2. Defining Schemas with Pydantic

Pydantic creates "contracts" for your agent's outputs:

```python
from pydantic import BaseModel, Field

class EmailContent(BaseModel):
    """Blueprint for email outputs"""
    subject: str = Field(
        description="Concise, descriptive subject line (5-10 words)"
    )
    body: str = Field(
        description="Formatted content with greeting, paragraphs, and signature"
    )
```

### Schema Enforcement:
1. **Type Checking**: Ensures `subject` and `body` are strings
2. **Description Guidance**: Helps the model understand requirements
3. **Validation**: Rejects malformed outputs automatically

> Note: Install Pydantic first: `pip install pydantic`

## 3. Generating JSON Outputs with LlmAgent

Tie your schema to an agent:

```python
from google.adk.agents import LlmAgent

email_agent = LlmAgent(
    name="email_agent",
    model="gemini-2.0-flash",
    instruction="""
    Generate professional emails with:
    - Clear subject line
    - Proper greeting/closing
    - Well-structured body
    Response MUST be JSON matching EmailContent schema
    """,
    output_schema=EmailContent,  # Your Pydantic model
    output_key="email",         # Nest under this key
)
```

### Expected Output Format:
```json
{
  "email": {
    "subject": "Team Meeting",
    "body": "Hi Team..."
  }
}
```

Key Parameters:
- `output_schema`: Your Pydantic model
- `output_key`: Top-level key for the structured data
- `model`: Determines output quality (gemini-2.0-flash recommended)

## 4. Todo List Agent Example

### Different Schema, Same Pattern

```python
from datetime import date

class TodoItem(BaseModel):
    task: str = Field(description="Task name")
    due_date: date = Field(description="YYYY-MM-DD format")
    priority: str = Field(description="low/medium/high")

todo_agent = LlmAgent(
    name="todo_agent",
    model="gemini-2.0-flash",
    instruction="Generate todo items in exact JSON format",
    output_schema=TodoItem,
    output_key="todo"
)

# Example usage
todo = todo_agent.run("Create high-priority task to call client today")
print(f"Task: {todo['todo']['task']}")
print(f"Due: {todo['todo']['due_date']}")
```

### Expected Output
```json
{
  "todo": {
    "task": "Call client",
    "due_date": "2023-10-13",
    "priority": "high"
  }
}
```

## 6. Pro Tips & Troubleshooting

### Best Practices
1. **Clear Instructions**:
   ```python
   instruction="Response MUST be valid JSON with..."
   ```
2. **Model Selection**:
   - `gemini-2.0-flash`: Fast & structured
   - `gemini-2.0-pro`: Complex outputs
3. **Validation**:
   ```python
   try:
       EmailContent(**response["email"])
   except ValueError as e:
       print("Invalid output:", e)
   ```

### Common Issues
- **Missing Fields**: Add clearer schema descriptions
- **Format Errors**: Specify date/string formats in Field()
- **Extra Text**: Use `DO NOT include explanations` in instructions

## Next Steps

1. Experiment with your own schemas
2. Try integrating with APIs using the structured outputs
3. Explore [ADK Documentation](https://google.github.io/adk-docs/) for advanced features

---
**Structured outputs make your agents reliable and production-ready!**